# Operationalize Deep Learning Models with Azure CLI 2.0

In this example, we show how to deploy the trained Keras (tensorflow) model as a web service through [Azure CLI](https://azure.microsoft.com/en-us/blog/announcing-general-availability-of-vm-storage-and-network-azure-cli-2-0/). The target machine learning problem is [CIFAR-10 - Object Recognition in Images](https://www.kaggle.com/c/cifar-10).

Specifically, we train a Keras network and evaluate the trained model on testing data. In Section Model Training, we show the steps to save a Keras model, deploy the model through Azure CLI, and test the deployed web service.

## Environment setup 

In this example, we use Deep Learning Virtual Machine - Linux OS, Standard [NC6 (6 vcpus, 56 GB memory) machine](https://azure.microsoft.com/en-us/blog/azure-n-series-preview-availability/) as the compute resource, where we train and deploy the model. Other types of Azure Linux VM should work as well.

We use following tools on this DLVM:
    - Python 3
    - Jupyter Notebook 
    - Azure CLI 

From your local machine ssh into the VM. In VM's CLI console, clone the repo to a desired location using below command. 
       
       $ git clone https://github.com/FrancescaLazzeri/DLDeploymentAzureCLI.git
        
Launch Jupyter Server manually. It matters where and how the Jupyter server is launched. In order to make the remaining tutorial smooth, please follow below steps to launch Jupyter server. In VM's CLI console, (1) switch to use `py35` conda environment, and (2) launch Jupyter server from `py35` conda environment manually. Please follow below instructions to achieve it. 

    1) Change the conda environment 
        $ source activate py35 

    2) Start the jupyter notebook from the command line
        $ jupyter notebook --no-browser --ip=* 
   
    3) In the output of above command, locate `http(s)://[all ip addresses on your system]:port_number/`. An example is, `https://[all ip addresses on your system]:9999/`. If it has `token` shown in part of this address, it is very likely you need to finish steps 7) - 10). If there is no URL in the output. Instead, you see: error message `No such notebook dir: "/dsvm/Notebooks"`, follow this page [Access Jupyter notebooks on Azure DSVM](http://yiyujia.blogspot.com/2018/05/access-jupyter-notebooks-on-azure-dsvm.html) to set c.NotebookApp.notebook_dir to be your home directory in `jupyter_notbook_config.py` file.

    4) On the Azure portal create an inbound rule for port `port_number` (e.g. 9999 in above example), if not an inbound rule has been created for this port before. (To create an inbound rule: at the VM's overview page on Azure portl, click `Networking` and check if section `INBOUND PORT RULES` already has 9999 shown in PORT column. If not, click `Add inbound port rule`, then set `Destination port ranges` to be 9999, and modify `Name` to be "Port_9999". Finally click `Add`).
    
    5) On your local machine, open a web-browser and go to `http(s)://<VM IP address>:port_number/` as indicated in the above command output. When prompted for password, enter the password you have previously created. If you don't remember the password, finish steps 7) - 10). 
    
    6) If you succesfully access `http(s)://<VM IP address>:port_number/` (ignore `Certificate error`), skip remaining steps. Otherwise, continue on below steps. 
    
    7) In the concole, shut down the Jupyter server by typing `Ctrl+C`.
    
    8) Generate a jupyter configuration file.
        $ jupyter notebook --generate-config 
    
    9) Generate a jupyter password (create a password and enter it twice).
        $ jupyter notebook password 
  
    10) Start the jupyter notebook from the command line and go to step 5). 
        $ jupyter notebook --no-browser --ip=* 

By now, you should have the Jupyter server on and you can access the notebooks from the local copy of repo. Open the notebook `OperationalizeDLModelswithAMLCLI.ipynb` and make sure the kernel is set to be `Python [conda env: py35]`(To change a kernel, choose `kernel` - `Change kernel` in notebook menu).

### Check configuration from Jupyter notebooks

As a prerequisite, Azure CLI must be configured to make a set of "az ml" commands work for [model management](https://docs.microsoft.com/en-us/azure/machine-learning/preview/model-management-cli-reference). In a notebook cell, type `az -h` and `az ml -h`. The output should look similar as below screenshots. This check ensures relevant commands works in the [Deploy model as a Web Service](#deploy_model) section. 

## Import the necessary libraries 
In this section, we will show you how to import the necessary libraries.

In [23]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from common.params import *
from common.utils import *

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
# Performance Improvement
# 1. Make sure channels-first (not last)
K.backend.set_image_data_format('channels_first')

In [6]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.4 |Anaconda custom (64-bit)| (default, Nov 20 2017, 18:44:38) 
[GCC 7.2.0]
Keras:  2.1.4
Numpy:  1.14.1
Tensorflow:  1.6.0
tensorflow
channels_first
GPU:  ['Tesla K80']
CUDA Version 9.0.176
CuDNN Version  7.0.5


## Model training 

In [7]:
def create_symbol(n_classes=N_CLASSES):
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu',
                     input_shape=(3, 32, 32)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [8]:
def init_model(m, lr=LR, momentum=MOMENTUM):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(lr, momentum),
        metrics = ['accuracy'])
    return m

In [9]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Data does not exist. Downloading https://ikpublictutorial.blob.core.windows.net/deeplearningframeworks/cifar-10-python.tar.gz
Extracting files...
Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 2.73 s, sys: 1.12 s, total: 3.85 s
Wall time: 8.31 s


In [10]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 437 ms, sys: 2.63 s, total: 3.07 s
Wall time: 5.12 s


In [11]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 28.5 ms, sys: 359 µs, total: 28.9 ms
Wall time: 28 ms


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 32, 32)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 32, 32)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 16, 16)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 16, 16)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 100, 8, 8)         0         
__________

In [13]:
%%time
# Main training loop: 1m16s
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/10
50000/50000 [==============================] - 21s 420us/step - loss: 1.8471 - acc: 0.3300
Epoch 2/10
50000/50000 [==============================] - 20s 393us/step - loss: 1.4364 - acc: 0.4761
Epoch 3/10
50000/50000 [==============================] - 20s 392us/step - loss: 1.2250 - acc: 0.5598
Epoch 4/10
50000/50000 [==============================] - 20s 394us/step - loss: 1.0637 - acc: 0.6220
Epoch 5/10
50000/50000 [==============================] - 20s 394us/step - loss: 0.9402 - acc: 0.6680
Epoch 6/10
50000/50000 [==============================] - 20s 395us/step - loss: 0.8518 - acc: 0.6993
Epoch 7/10
50000/50000 [==============================] - 20s 394us/step - loss: 0.7687 - acc: 0.7296
Epoch 8/10
50000/50000 [==============================] - 20s 396us/step - loss: 0.7046 - acc: 0.7530
Epoch 9/10
50000/50000 [==============================] - 20s 393us/step - loss: 0.6471 - acc: 0.7700
Epoch 10/10
50000/50000 [==============================] - 20s 392us/step - loss: 

In [14]:
%%time
# Main evaluation loop
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 677 ms, sys: 50.6 ms, total: 728 ms
Wall time: 1.48 s


In [15]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.7639


## Save the model

In [16]:
#get the current working directory
print(os.getcwd()) 

#list files in current working directory
# os.listdir(os.curdir)

/home/mylogin/notebooks/DeepLearningModelDeployment


In [24]:
local_path = os.getcwd()
o16n_path = os.path.join(local_path,'o16n')  
model_path = os.path.join(o16n_path,'kerastfmodel')
model_file_name = os.path.join(model_path,'kerastfmodel.h5')
score_file_name = os.path.join(model_path, 'score.py')


if not os.path.exists(local_path):
    os.makedirs(local_path)
if not os.path.exists(o16n_path):
    os.makedirs(o16n_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)

In [18]:
print(model_path)
print(score_file_name)

/home/mylogin/notebooks/DeepLearningModelDeployment/o16n/kerastfmodel
/home/mylogin/notebooks/DeepLearningModelDeployment/o16n/kerastfmodel/score.py


In [19]:
# Save the Model
model.save(model_file_name)

### Write a scoring script 
In order to create a web service, you will create a scoring script that will load the models, perform the prediction, and return the result. 
Azure ML uses init() and run() functions inside this scoring script for that purpose:

   - The init() function initializes the web service and loads the saved model. 
   - The run() function uses the model and the input data to return a prediction which is executed on a scoring call.

In [20]:
%%writefile $score_file_name

import numpy as np
import os
import sys
import keras as K
from io import BytesIO
from PIL import Image, ImageOps
import base64
import json

def init():
    
    global model  

    print("Executing init() method...")
    print("Python version: " + str(sys.version) + ", keras version: " + K.__version__)
    # Load the model 
    model = K.models.load_model('kerastfmodel.h5')
    return


def run(inputString):
    
    responses = []
    base64Dict = json.loads(inputString)

    for k, v in base64Dict.items():
        img_file_name, base64Img = k, v
    decoded_img = base64.b64decode(base64Img)
    img_buffer = BytesIO(decoded_img)
    imageData = Image.open(img_buffer).convert("RGB")

    # Evaluate the model using the input data
    img = ImageOps.fit(imageData, (32, 32), Image.ANTIALIAS)
    img_conv = np.array(img) # shape: (32, 32, 3)
    # Scale pixel intensity
    x_test = img_conv / 255.0
    # Reshape
    x_test = np.moveaxis(x_test, -1, 0)
    x_test = np.expand_dims(x_test, 0)  # shape (1, 3, 32, 32)

    y_pred = model.predict(x_test)
    y_pred = np.argmax(y_pred, axis=-1)
    # print(y_pred)
    LABELS = ["airplane", "automobile", "bird", "cat", "deer", "fog", "frog", "horse", "ship", "truck"]
    resp = {img_file_name: str(LABELS[y_pred[0]])}

    responses.append(resp)
    return json.dumps(responses)
    
  
if __name__ == "__main__":
    init()
    # input data
    img_path = 'automobile8.png'
    encoded = None
    with open(img_path, 'rb') as file:
      encoded = base64.b64encode(file.read())
    img_dict = {img_path: encoded.decode('utf-8')}
    body = json.dumps(img_dict)
    resp = run(body)
    print(resp)

Writing /home/mylogin/notebooks/DeepLearningModelDeployment/o16n/kerastfmodel/score.py


## Model deployment as web service

In this section, we show the steps to deploy the previously trained model on [Azure Container Service (ACS)](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/microsoft.acs) via [Azure CLI](https://azure.microsoft.com/en-us/blog/announcing-general-availability-of-vm-storage-and-network-azure-cli-2-0/). 

During this section, following Azure resources will be created:
    
    - Resource group defined in variable YOUR_RESOURCE_GROUP
        * Machine Learning Model Management
        * cluster environment (Microsoft.MachineLearningCompute/operationalizationClusters)
    - Resource group created during the cluster environment provision (YOUR_RESOURCE_GROUP plus"-azureml-xxxxx") 
        * Container registry
        * Container service
        * .... a bunch of other automatically provisoned resources

In [26]:
# Copy conda_dependencies.yml file into the directory model_path. This file is needed for model deployment.
import shutil
yml_file_name = os.path.join(local_path,'common/conda_dependencies.yml')
shutil.copyfile(yml_file_name, os.path.join(model_path, os.path.split(yml_file_name)[1]))

'/home/mylogin/notebooks/DeepLearningModelDeployment/o16n/kerastfmodel/conda_dependencies.yml'

In [27]:
#list files in current working directory
os.listdir(os.curdir)

['conda_dependencies.yml', 'score.py', 'kerastfmodel.h5', 'automobile8.png']

In [ ]:
# login your Azure account and set Azure subscription
!az login

When executing `!az login`, you will see following output message: To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code xxxxxxx to authenticate. Finish this login task in a browser and close the browser page.

In the output of this command, you should see following information about your Azure subscription. Locate the "name" field and use it as the SUB_Name in the next cell. If you have more than one Azure subscritons, please choose one for this execercise. 

      {
        "cloudName": "AzureCloud",
        "id": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
        "isDefault": true,
        "name": "Your Subscription Name",
        "state": "Enabled",
        "tenantId": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
        "user": {
          "name": "yourloginEmail@xxx.com",
          "type": "user"
        }
      }


In [36]:
# Create Variable Names for Deployment
SUB_Name = '"Your Azure Subscription name"' # Make sure you include double quote for your Azure subscription name
LOCATION = 'Pick an Azure Region Here!! e.g. eastus2' # resource location
DEPLOY_NAME = 'Pick a Deployment Name Here!!' # limite DEPLOY_NAME to 14 characters or less
YOUR_RESOURCE_GROUP = DEPLOY_NAME + 'rg' 
MM_ACCOUNT = DEPLOY_NAME + 'mmacc'     
CLUSTER_SERVICE_NAME = DEPLOY_NAME + 'clus' + 'srvc'
CLUSTER_ENV_NAME = DEPLOY_NAME + 'clus' + 'env'

In [8]:
!az account set -s $SUB_Name

In [1]:
# Create a resource group (optinal, only if you are about to create a new resource group)
!az group create --l $LOCATION --name $YOUR_RESOURCE_GROUP

In [2]:
# execute following command to create a Machine Learning Model Management resource in the resource group
!az ml account modelmanagement create -l $LOCATION -n $MM_ACCOUNT -g $YOUR_RESOURCE_GROUP

In [3]:
# Execute fllowing command to create the cluster environment (Microsoft.MachineLearningCompute/operationalizationClusters)
# !az ml env setup --cluster -l $LOCATION -n $CLUSTER_ENV_NAME -g $YOUR_RESOURCE_GROUP
print('az ml env setup --cluster -l ', LOCATION, '-n ', CLUSTER_ENV_NAME,'-g ', YOUR_RESOURCE_GROUP)

Execute the above command in a new CLI console. (You need to ssh into the VM from your local machine again, as the Jupyter server is still running from previous openned console.) At the prompt, type 'n' for question "Reuse storage and ACR (Y/n)?" and 'Y' for question "Continue with this subscription (Y/n)?".

In [4]:
# You can check the information on the environment with the following command. 
!az ml env show -g $YOUR_RESOURCE_GROUP -n $CLUSTER_ENV_NAME

In [5]:
!az ml account modelmanagement set -n $MM_ACCOUNT -g $YOUR_RESOURCE_GROUP

In [6]:
!az ml env set -g $YOUR_RESOURCE_GROUP -n $CLUSTER_ENV_NAME

In [25]:
# Change the current working directory to model_path
# os.chdir(model_path)
# list files in current working directory
# Make sure current directory contains 'score.py', 'kerastfmodel.h5', and 'conda_dependencies.yml'

os.listdir(os.curdir)

['conda_dependencies.yml', 'score.py', 'kerastfmodel.h5', 'automobile8.png']

In [7]:
# create web service - this step can take several minutes
!az ml service create realtime --model-file kerastfmodel.h5 -f score.py -n $CLUSTER_SERVICE_NAME -r python -c conda_dependencies.yml

When the web service is succesfully deployed, you should see a field "Service ID" from the output information. 
Take this information and put in CLUSTER_SERVICE_ID in the next cell.

In [15]:
CLUSTER_SERVICE_ID = 'Your cluster service Id here!!'